In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scanpy as sc
import scib

In [3]:
import sys
sys.path.append("../../../autotalker")

In [4]:
from autotalker.benchmarking import compute_clisis, compute_cas

In [213]:
adata_auto = sc.read_h5ad("../../datasets/srt_data/gold/results/seqfish_mouse_organogenesis_autotalker_oneshot_integrated.h5ad")
model = "autotalker"
adata = adata_auto

In [201]:
adata_graph = sc.read_h5ad("../../datasets/srt_data/gold/results/seqfish_mouse_organogenesis_graphst_oneshot_integrated.h5ad")
model = "graphst"
adata = adata_graph

In [6]:
# Gene Program Mask
nichenet_keep_target_genes_ratio = 0.01
nichenet_max_n_target_genes_per_gp = 25344
include_mebocost_gps = True
mebocost_species = "mouse"
gp_filter_mode = "subset"
combine_overlap_gps = True
overlap_thresh_source_genes = 0.9
overlap_thresh_target_genes = 0.9
overlap_thresh_genes = 0.9

# Data
dataset = "starmap_plus_mouse_cns"
reference_batches = ["batch1_sample", "batch2_sample", "batch3_sample"]
counts_key = "counts"
condition_key = "batch"
n_neighbors = 12
spatial_key = "spatial"
adj_key = "spatial_connectivities"
mapping_entity_key = "mapping_entity"
filter_genes = True
n_hvg = 2000
gp_targets_mask_key = "autotalker_gp_targets"
gp_sources_mask_key = "autotalker_gp_sources"
gp_names_key = "autotalker_gp_names"

# Model
load_timestamp = "27032023_184359"
latent_key = "latent"

# Benchmarking
cell_type_key = "cell_type"
spatial_knng_key = "autotalker_spatial_knng"
latent_knng_key = "autotalker_latent_knng"

# Other
random_seed = 0

In [214]:
# Compute metrics
metrics_dict = {}

# Spatial conservation metrics
metrics_dict["cas"] = compute_cas(
    adata=adata,
    cell_type_key=cell_type_key,
    condition_key=condition_key,
    spatial_knng_key=spatial_knng_key,
    latent_knng_key=latent_knng_key,
    spatial_key=spatial_key,
    latent_key=f"{model.lower()}_{latent_key}_run1")
metrics_dict["clisis"] = compute_clisis(
    adata=adata,
    cell_type_key=cell_type_key,
    condition_key=condition_key,
    spatial_knng_key=spatial_knng_key,
    latent_knng_key=latent_knng_key,
    spatial_key=spatial_key,
    latent_key=f"{model.lower()}_{latent_key}_run1")

# Batch correction metrics
metrics_dict["asw"] = scib.me.silhouette_batch(
    adata=adata,
    batch_key=condition_key,
    label_key=cell_type_key,
    embed=f"{model.lower()}_{latent_key}_run1")
metrics_dict["ilisi"] = scib.me.ilisi_graph(
    adata=adata,
    batch_key=condition_key,
    type_="embed",
    use_rep=f"{model.lower()}_{latent_key}_run1")
    #type_="knn")

Computing spatial nearest neighbor graph for batch embryo1_z2...
Computing spatial nearest neighbor graph for batch embryo1_z5...
Computing spatial nearest neighbor graph for batch embryo2_z2...
Computing spatial nearest neighbor graph for batch embryo2_z5...
Computing spatial nearest neighbor graph for batch embryo3_z2...
Computing spatial nearest neighbor graph for batch embryo3_z5...
Combining spatial nearest neighbor graphs...
Computing spatial neighborhood enrichment scores...
Computing latent nearest neighbor graph...
Computing latent neighborhood enrichment scores...
Computing CAS...
Using precomputed spatial nearest neighbor graph...
Computing spatial cell CLISI scores for entire dataset...
Using precomputed latent nearest neighbor graph...
Computing latent cell CLISI scores...
Computing CLISIS...
mean silhouette per group:                                 silhouette_score
group                                           
Allantois                               0.933600
Anterior 

In [215]:
# Autotalker new (2 layers)
metrics_dict

{'cas': 0.5707141417257142,
 'clisis': 0.9130278398086543,
 'asw': 0.9288969028701565,
 'ilisi': 0.15331887289540216}

In [209]:
# Autotalker new
metrics_dict

{'cas': 0.55854807564555,
 'clisis': 0.9077236380646184,
 'asw': 0.9264081005211691,
 'ilisi': 0.2305902955803667}

In [200]:
# Autotalker old
metrics_dict

{'cas': 0.5418271019673746,
 'clisis': 0.9109334988770585,
 'asw': 0.9459392645800031,
 'ilisi': 0.1749282196392648}

In [203]:
# GraphST
metrics_dict

{'cas': 0.5346951918608724,
 'clisis': 0.9086096795728897,
 'asw': 0.9568973614685743,
 'ilisi': 0.21278161156319317}

In [ ]:
adata.obsp["autotalker_latent_run1_connectivities"].sum(axis=1)

In [ ]:
sc.pp.neighbors(adata=adata,
                # use_rep=spatial_key,
                n_neighbors=15,
                random_state=0,
                key_added="spatial",
                knn=True)

In [ ]:
adata.obsp["spatial_connectivities"].toarray()[:10, :200]

In [18]:
batches = adata.obs[condition_key].unique().tolist()[:7]

In [28]:
adata_batch_list = []
for batch in batches:
    adata_batch_list.append(adata[adata.obs[condition_key] == batch][:50, :].copy())
adata = sc.concat(adata_batch_list, join="inner")

adata.obs["cell_type"] = adata.obs["cell_type"].astype("category")

In [175]:
adata

AnnData object with n_obs × n_vars = 52568 × 351
    obs: 'cell_type', 'batch', 'reference'
    uns: 'graphst_latent_run1', 'graphst_latent_run10', 'graphst_latent_run3', 'graphst_latent_run4', 'graphst_latent_run5', 'graphst_latent_run6', 'graphst_latent_run7', 'graphst_latent_run8', 'graphst_latent_run9', 'graphst_model_training_duration_run1', 'graphst_model_training_duration_run10', 'graphst_model_training_duration_run3', 'graphst_model_training_duration_run4', 'graphst_model_training_duration_run5', 'graphst_model_training_duration_run6', 'graphst_model_training_duration_run7', 'graphst_model_training_duration_run8', 'graphst_model_training_duration_run9', 'umap'
    obsm: 'graphst_latent_run1', 'graphst_latent_run10', 'graphst_latent_run10_X_umap', 'graphst_latent_run1_X_umap', 'graphst_latent_run3', 'graphst_latent_run3_X_umap', 'graphst_latent_run4', 'graphst_latent_run4_X_umap', 'graphst_latent_run5', 'graphst_latent_run5_X_umap', 'graphst_latent_run6', 'graphst_latent_run6_X_

In [ ]:
adata_sample = adata[:5, :]

In [ ]:
adata_sample = adata_batch_list[0].copy()

In [ ]:
adata_sample.obsp["spatial_connectivities"].toarray()

In [ ]:
import squidpy as sq
sq.gr.nhood_enrichment(adata=adata_sample,
                       cluster_key="cell_type",
                       connectivity_key="spatial",
                       n_perms=1000,
                       seed=0,
                       show_progress_bar=False)

In [182]:
# Compute metrics
metrics_dict = {}

# Spatial conservation metrics
metrics_dict["cas"] = compute_cas(
    adata=adata,
    cell_type_key=cell_type_key,
    condition_key=condition_key,
    spatial_knng_key=spatial_knng_key,
    latent_knng_key=latent_knng_key,
    spatial_key=spatial_key,
    latent_key=f"{model.lower()}_{latent_key}_run1")
"""
metrics_dict["clisis"] = compute_clisis(
    adata=adata,
    cell_type_key=cell_type_key,
    condition_key=condition_key,
    spatial_knng_key=spatial_knng_key,
    latent_knng_key=latent_knng_key,
    spatial_key=spatial_key,
    latent_key=f"{model.lower()}_{latent_key}_run1")

# Batch correction metrics
metrics_dict["asw"] = scib.me.silhouette_batch(
    adata=adata,
    batch_key=condition_key,
    label_key=cell_type_key,
    embed=f"{model.lower()}_{latent_key}_run1")
metrics_dict["ilisi"] = scib.me.ilisi_graph(
    adata=adata,
    batch_key=condition_key,
    type_="embed",
    use_rep=f"{model.lower()}_{latent_key}_run1")
    #type_="knn")
"""

Computing spatial nearest neighbor graph for batch embryo1_z2...
Computing spatial nearest neighbor graph for batch embryo1_z5...
Computing spatial nearest neighbor graph for batch embryo2_z2...
Computing spatial nearest neighbor graph for batch embryo2_z5...
Computing spatial nearest neighbor graph for batch embryo3_z2...
Computing spatial nearest neighbor graph for batch embryo3_z5...
Combining spatial nearest neighbor graphs...
Computing spatial neighborhood enrichment scores...
Computing latent nearest neighbor graph...
Computing latent neighborhood enrichment scores...
22101.322463510936
____________________________________
19581.100115731937
[[ 3.81817995e+02 -1.43669233e+01 -1.35434606e+00 -2.42815344e+01
  -2.94425195e+01 -8.70529537e+00 -2.22849003e+01 -6.86531409e+00
   1.89040958e+00 -1.08856663e+00 -5.06420428e+01 -3.26262151e+01
   3.16328763e+01 -9.31856026e-01  1.78688831e+01  6.31806914e+01
  -7.44089192e+00 -2.62959243e+01 -1.71328431e+01 -1.86438397e+01
  -3.29692899e

'\nmetrics_dict["clisis"] = compute_clisis(\n    adata=adata,\n    cell_type_key=cell_type_key,\n    condition_key=condition_key,\n    spatial_knng_key=spatial_knng_key,\n    latent_knng_key=latent_knng_key,\n    spatial_key=spatial_key,\n    latent_key=f"{model.lower()}_{latent_key}_run1")\n\n# Batch correction metrics\nmetrics_dict["asw"] = scib.me.silhouette_batch(\n    adata=adata,\n    batch_key=condition_key,\n    label_key=cell_type_key,\n    embed=f"{model.lower()}_{latent_key}_run1")\nmetrics_dict["ilisi"] = scib.me.ilisi_graph(\n    adata=adata,\n    batch_key=condition_key,\n    type_="embed",\n    use_rep=f"{model.lower()}_{latent_key}_run1")\n    #type_="knn")\n'